In [1]:
from CNN.cnn_utils import get_train_data
from LSTM.lstm_autoencoder import LSTM_AE
from LSTM.lstm_autoencoders_utils import train_lstm_autoencoder
from RF_XGB.RandomForest import train_random_forest
from models_utils.Datasets import *
from models_utils.GLOBALS import *
from models_utils.utils import get_results

In [2]:
# get train data
train_data = pd.read_csv('train.csv')
data_type_1 = train_data[train_data['sensor'] == 'vicon']['id'].tolist()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch']['id'].tolist()

In [3]:
# sizes of padding/cutting
embedding_size = 128
target_size_type1 = 4000
target_size_type2 = 1350

In [4]:
# train or load models
train_or_load_autoencoders = 'load'
if train_or_load_autoencoders == 'train':
    Type1LSTMAutoencoder = train_lstm_autoencoder(data_type_1, '1', target_size_type1, embedding_size, 0.000004, 64, 15)
    Type2LSTMAutoencoder = train_lstm_autoencoder(data_type_2, '2', target_size_type2, embedding_size, 0.000004, 64, 15)
elif train_or_load_autoencoders == 'load':
    Type1LSTMAutoencoder = LSTM_AE(target_size_type1, 3, embedding_size).to(device)
    Type1LSTMAutoencoder.load_state_dict(torch.load('Type1LSTMAutoencoder.pth'))
    Type2LSTMAutoencoder = LSTM_AE(target_size_type2, 3, embedding_size).to(device)
    Type2LSTMAutoencoder.load_state_dict(torch.load('Type2LSTMAutoencoder.pth'))
else:
    raise ValueError('Wrong train or load')

In [5]:
# calculate or load train data
embedding_names = [f'embedding_feature_{i + 1}' for i in range(18)]
calculate_or_load_train_data = 'calculate'
if calculate_or_load_train_data == 'calculate':
    data_type_1, data_type_2 = get_train_data('EmbeddingRF', Type1LSTMAutoencoder, Type2LSTMAutoencoder,
                                              target_size_type1, target_size_type2, is_autoencoder=True)
elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('EmbeddingRF_type1.csv')
    data_type_2 = pd.read_csv('EmbeddingRF_type2.csv')

In [ ]:
# train rf
cols_to_drop_type_1 = ['activity', 'dominant_freq_y', 'count_x', 'count_y', 'count_z', 'dominant_freq_x']
rf_classifier_type1, label_encoder_type1 = train_random_forest(data_type_1, cols_to_drop_type_1, n_estimators=200)
cols_to_drop_type_2 = ['activity', 'dominant_freq_y', 'count_x', 'count_y', 'count_z', 'dominant_freq_x']
rf_classifier_type2, label_encoder_type2 = train_random_forest(data_type_2, cols_to_drop_type_2, n_estimators=200)

In [ ]:
# get and save results for test data
results = get_results(Type1LSTMAutoencoder, Type2LSTMAutoencoder, rf_classifier_type1, rf_classifier_type2,
                      label_encoder_type1, label_encoder_type2, target_size_type1, target_size_type2,
                      cols_to_drop_type_1, cols_to_drop_type_2, embedding_names)
results.to_csv('results_Embedding_RF.csv', index=False)